In [15]:
# Install PostgreSQL
!apt-get -qq install postgresql postgresql-contrib

# Start the PostgreSQL service
!service postgresql start

# Set the default username and password (Colab defaults to "postgres")
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'yourpassword';"


 * Starting PostgreSQL 14 database server
   ...done.
ALTER ROLE


In [16]:
!pip install sqlalchemy psycopg2-binary

In [ ]:
# Create database aml_database
!sudo -u postgres psql -c "CREATE DATABASE aml_database;"


CREATE DATABASE


In [ ]:
import psycopg2

# Connecting to PostgreSQL Database
conn = psycopg2.connect(
    dbname="aml_database",
    user="postgres",
    password="yourpassword",  # Fill in your password here
    host="localhost"
)
cursor = conn.cursor()

# Test the connection
cursor.execute("SELECT version();")
print(cursor.fetchone())

# Close the connection
cursor.close()
conn.close()


('PostgreSQL 14.15 (Ubuntu 14.15-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [20]:
# Reestablishing a connection
conn = psycopg2.connect(
    dbname="aml_database",
    user="postgres",
    password="yourpassword",
    host="localhost"
)
cursor = conn.cursor()

# Create Tables
create_tables_query = '''
-- Customer Table
DROP TABLE IF EXISTS Transaction;
DROP TABLE IF EXISTS Alert;
DROP TABLE IF EXISTS Account;
DROP TABLE IF EXISTS Customer;
CREATE TABLE Customer (
    customer_id VARCHAR(50) PRIMARY KEY,
    customer_type VARCHAR(50) NOT NULL,
    customer_name VARCHAR(100) NOT NULL,
    customer_line_of_business VARCHAR(100),
    customer_expected_products TEXT,
    customer_expected_geographies TEXT,
    customer_incorporation_residence_country VARCHAR(10) NOT NULL
);

-- Account Table
DROP TABLE IF EXISTS Account;
CREATE TABLE Account (
    account_id VARCHAR(50) PRIMARY KEY,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE CASCADE,
    date_of_opening DATE NOT NULL,
    expected_incoming_activity NUMERIC(15,2),
    expected_outgoing_activity NUMERIC(15,2)
);

-- Alert Table
DROP TABLE IF EXISTS Alert;
CREATE TABLE Alert (
    detection_id VARCHAR(50) PRIMARY KEY,   -- 将 detection_id 设为主键
    alert_id VARCHAR(50) NOT NULL,          -- alert_id 不再是唯一，但必须存在
    alert_date DATE NOT NULL,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE SET NULL,
    rule_name VARCHAR(255) NOT NULL,
    alerted_transactions TEXT,
    false_positive_true_positive VARCHAR(20),
    alert_narrative TEXT
);

-- Transaction Table
DROP TABLE IF EXISTS Transaction;
CREATE TABLE Transaction (
    transaction_id VARCHAR(50) PRIMARY KEY,
    transaction_date DATE NOT NULL,
    transaction_type VARCHAR(100) NOT NULL,
    customer_id VARCHAR(50) REFERENCES Customer(customer_id) ON DELETE SET NULL,
    account_id VARCHAR(50) REFERENCES Account(account_id) ON DELETE CASCADE,
    incoming_outgoing VARCHAR(20) CHECK (incoming_outgoing IN ('Incoming', 'Outgoing')),
    amount NUMERIC(15,2) NOT NULL,
    originator VARCHAR(100),
    originator_country VARCHAR(10),
    beneficiary VARCHAR(100),
    beneficiary_country VARCHAR(10)
);
'''

# Execute the SQL statement to create the table
cursor.execute(create_tables_query)
conn.commit()
print("All tables created successfully!")

# Close the connection
cursor.close()
conn.close()

All tables created successfully!


In [21]:
import pandas as pd

# Reestablishing a connection
conn = psycopg2.connect(
    dbname="aml_database",
    user="postgres",
    password="yourpassword",
    host="localhost"
)
cursor = conn.cursor()

# ------------------- 1. Import Customer table ------------------- #
print("Starting import of Customer data...")
customer_df = pd.read_csv("/content/Customer Table.csv")
# Remove duplicate customer_id
customer_df = customer_df.drop_duplicates(subset=["Customer ID"])

for _, row in customer_df.iterrows():
    cursor.execute(
        "INSERT INTO Customer (customer_id, customer_type, customer_name, customer_line_of_business, customer_expected_products, customer_expected_geographies, customer_incorporation_residence_country) VALUES (%s, %s, %s, %s, %s, %s, %s)",
        (row["Customer ID"], row["Customer Type"], row["Customer Name"], row["Customer Line of Business"], row["Customer Expected Products"], row["Customer Expected Geographies"], row["Customer Incorporation/Residence Country"])
    )

conn.commit()
print("Customer data import completed!")

# ------------------- 2. Import Account table ------------------- #
print("\nStart importing Account data...")
account_df = pd.read_csv("/content/Account Table.csv")
# Remove duplicate account_id
account_df = account_df.drop_duplicates(subset=["Account ID"])

# Import only customer_ids that exist in the Customer table
cursor.execute("SELECT customer_id FROM Customer")
valid_customers = {row[0] for row in cursor.fetchall()}

for _, row in account_df.iterrows():
    if row["Customer ID"] in valid_customers:
        cursor.execute(
            "INSERT INTO Account (account_id, customer_id, date_of_opening, expected_incoming_activity, expected_outgoing_activity) VALUES (%s, %s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s) ON CONFLICT (account_id) DO NOTHING",
            (row["Account ID"], row["Customer ID"], row["Date of Opening"], row["Expected Incoming Activity"], row["Expected Outgoing Activity"])
        )
conn.commit()
print("Account data import completed!")

# ------------------- 3. Import Alert table ------------------- #
print("\nStart importing Alert data...")
alert_df = pd.read_csv("/content/Alert Table.csv")
# Remove duplicate detection_id (since it is now the primary key)
alert_df = alert_df.drop_duplicates(subset=["Detection ID"])

# customer_id is allowed to be NULL, but if present, it must be in the Customer table
for _, row in alert_df.iterrows():
    customer_id = row["Customer ID"] if pd.notna(row["Customer ID"]) and row["Customer ID"] in valid_customers else None
    cursor.execute(
        "INSERT INTO Alert (detection_id, alert_id, alert_date, customer_id, rule_name, alerted_transactions, false_positive_true_positive, alert_narrative) VALUES (%s, %s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s, %s, %s, %s) ON CONFLICT (detection_id) DO NOTHING",
        (row["Detection ID"], row["Alert ID"], row["Alert Date"], customer_id, row["Rule Name"], row["Alerted Transactions per Detection"], row["False Positive / True Positive"], row["Alert Narrative"])
    )
conn.commit()
print("Alert data import completed!")

# ------------------- 4. Import Transaction table ------------------- #
print("\nStart importing transaction data...")

transaction_df = pd.read_csv("/content/Transaction Table.csv")
# Remove duplicate transaction_id
transaction_df = transaction_df.drop_duplicates(subset=["Transaction ID"])

# Get a valid customer_id and account_id
cursor.execute("SELECT customer_id FROM Customer")
valid_customers = {row[0] for row in cursor.fetchall()}

cursor.execute("SELECT account_id FROM Account")
valid_accounts = {row[0] for row in cursor.fetchall()}

for _, row in transaction_df.iterrows():
    customer_id = row["Customer ID"] if pd.notna(row["Customer ID"]) and row["Customer ID"] in valid_customers else None
    account_id = row["Account"] if pd.notna(row["Account"]) and row["Account"] in valid_accounts else None

    if account_id is not None:  # account_id is required because it is a foreign key
        # Make sure all None are handled correctly
        transaction_values = (
            row["Transaction ID"],
            row["Transaction Date"],
            row["Transaction Type"],
            customer_id,
            account_id,
            row["Incoming/Outgoing"] if pd.notna(row["Incoming/Outgoing"]) else None,
            row["Amount"] if pd.notna(row["Amount"]) else 0,  # If the numeric column is empty, replace it with 0, or use None as needed
            row["Originator"] if pd.notna(row["Originator"]) else None,
            row["Originator Country"] if pd.notna(row["Originator Country"]) else None,
            row["Beneficiary"] if pd.notna(row["Beneficiary"]) else None,
            row["Beneficiary Country"] if pd.notna(row["Beneficiary Country"]) else None
        )

        cursor.execute(
            "INSERT INTO Transaction (transaction_id, transaction_date, transaction_type, customer_id, account_id, incoming_outgoing, amount, originator, originator_country, beneficiary, beneficiary_country) "
            "VALUES (%s, DATE '1900-01-01' + INTERVAL '1 day' * %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) "
            "ON CONFLICT (transaction_id) DO NOTHING",
            transaction_values
        )

conn.commit()
print("Transaction data import completed!")

# Close the database connection
cursor.close()
conn.close()
print("\nAll data has been successfully imported into the database!")

Starting import of Customer data...
Customer data import completed!

Start importing Account data...
Account data import completed!

Start importing Alert data...
Alert data import completed!

Start importing transaction data...
Transaction data import completed!

All data has been successfully imported into the database!


In [17]:
from sqlalchemy import create_engine
import pandas as pd

# Creating a SQLAlchemy Engine
db_user = "postgres"
db_password = "yourpassword"  # Replace with your password
db_host = "localhost"
db_name = "aml_database"

engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}")


In [22]:
# ------------------- 1. View the Customer table ------------------- #
print("\nViewing data from the Customer table...")
query = "SELECT * FROM Customer LIMIT 10;"
df_customer = pd.read_sql(query, engine)
display(df_customer)

# ------------------- 2. View the Account table ------------------- #
print("\nViewing data from the Account table...")
query = "SELECT * FROM Account LIMIT 10;"
df_account = pd.read_sql(query, engine)
display(df_account)

# ------------------- 3. View the Alert table ------------------- #
print("\nViewing data from the Alert table...")
query = "SELECT * FROM Alert LIMIT 10;"
df_alert = pd.read_sql(query, engine)
display(df_alert)

# ------------------- 4. View the Transaction table ------------------- #
print("\nViewing data of Transaction table...")
query = "SELECT * FROM Transaction LIMIT 10;"
df_transaction = pd.read_sql(query, engine)
display(df_transaction)

print("\nThe data of all tables has been successfully displayed!")



Viewing data from the Customer table...


,customer_id,customer_type,customer_name,customer_line_of_business,customer_expected_products,customer_expected_geographies,customer_incorporation_residence_country
0,C-1,Individual,John Diamond,Manufacturing,ACH; Wire,US,US
1,C-2,Business,RDF Plumbing,Plumbing Services,ACH; Wire; Cash Deposit; Internal Transfer,US,US
2,C-3,Individual,Kyle Strong,Service Industry,ACH; Wire; Cash Deposit; Internal Transfer,US; HK,HK
3,C-4,Business,JDF Industries,Oil refinement,ACH; Wire,US; SA,US



Viewing data from the Account table...


,account_id,customer_id,date_of_opening,expected_incoming_activity,expected_outgoing_activity
0,ACC-1,C-1,1980-03-03,100000.0,10000.0
1,ACC-2,C-2,2010-01-03,200000.0,200000.0
2,ACC-3,C-2,2024-02-17,200000.0,200000.0
3,ACC-4,C-3,2024-09-03,2000.0,2000.0
4,ACC-5,C-4,2007-07-04,10000000.0,10000000.0



Viewing data from the Alert table...


,detection_id,alert_id,alert_date,customer_id,rule_name,alerted_transactions,false_positive_true_positive,alert_narrative
0,A-1-1,A-1,2024-10-03,C-1,Cash Structuring $10k,"T-1, T-2, T-3, T-4, T-5, T-6, T-7",True Positive,No reasonable explanation for customer activit...
1,A-1-2,A-1,2024-10-03,C-1,Cash Structuring $10k,"T-7, T-8, T-9, T-10, T-11, T-12",True Positive,No reasonable explanation for customer activit...
2,A-1-3,A-1,2024-10-03,C-1,Rapid Movements of Funds,"T-1, T-2, T-3, T-4, T-5, T-6, T-7, T-8, T-9, T...",True Positive,No reasonable explanation for customer activit...
3,A-1-4,A-1,2024-10-03,C-1,Large Wire to High Risk Jursidiction,T-13,True Positive,No reasonable explanation for customer activit...
4,A-2-1,A-2,2024-10-03,C-2,Concentration Account,"T-14, T-15, T-16, T-17, T-20",True Positive,No reasonable explanation for customer activit...
5,A-2-2,A-2,2024-10-03,C-2,Rapid Movement of Funds,"T-14, T-15, T-16, T-17, T-20, T-21",True Positive,No reasonable explanation for customer activit...
6,A-3-1,A-3,2024-10-03,C-3,New Account Rule,"T-22, T-23, T-29",False Positive,Customer explains he worked as a dealer for a ...
7,A-3-2,A-3,2024-10-03,C-3,Cash Structuring $10k,"T-22, T-23",False Positive,Customer explains he worked as a dealer for a ...
8,A-5-1,A-5,2024-10-03,C-4,Large Wire to High Risk Jursidiction,"T-30, T-31, T-32, T-33",True Positive,A-5-1: No reasonable explanation for moving fu...
9,A-5-2,A-5,2024-10-03,C-4,Large Wire to High Risk Jursidiction,T-34,False Positive,A-5-1: No reasonable explanation for moving fu...



Viewing data of Transaction table...


,transaction_id,transaction_date,transaction_type,customer_id,account_id,incoming_outgoing,amount,originator,originator_country,beneficiary,beneficiary_country
0,T-1,2024-09-04,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
1,T-2,2024-09-05,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
2,T-3,2024-09-06,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
3,T-4,2024-09-07,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
4,T-5,2024-09-08,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
5,T-6,2024-09-09,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
6,T-7,2024-09-10,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
7,T-8,2024-09-11,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
8,T-9,2024-09-12,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US
9,T-10,2024-09-13,Cash Deposit,C-1,ACC-1,Incoming,9000.0,John Diamond,US,John Diamond,US



The data of all tables has been successfully displayed!
